## Homework 1 - Yeehaw! Data Wrangling!
### Your Name

**COLLABORATED WITH:_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _**

***
***
***

**Submit a PDF export of your notebook (100% PENALTY)** 

* Option 1 - File > Export Notebook to PDF
* Option 2 - File > Export Notebook to HTML (you cannot upload .html to Compass), open .html in browser, print to PDF
    - You need to use this option if you have math with `\begin{align} ... \end{align}`.

***
***
***

#### Table of Contents
- [Overview](#Overview)
- [[10 Points] Preliminaries](#[10-Points]-Preliminaries)
- [ACS](#ACS)
    - [[15 Points] Prep](#[15-Points]-Prep)
    - [[5 Points] Creating Keys](#[5-Points]-Creating-Keys)
    - [[15 Points] Grouped Features](#[15-Points]-Grouped-Features)
    - [[10 Points] Individual Features](#[10-Points]-Individual-Features)
    - [[5 Points] Labels](#[5-Points]-Labels)
- [[10 Points] FHFA](#[10-Points]-FHFA)
- [[25 Points] Transformations](#[25-Points]-Transformations)
- [[5 Points] Pickling](#[5-Points]-Pickling)

# Overview

**DOWNLOAD ALL DATA INTO THE SAME FOLDER**

We will be primarily using data from the 2016 American Community Survey (ACS) sourced from https://ipums.org/. 
You will need to **create a free account** to access the data.

ACS Steps:

* On the USA IPUMS page either click the Get Data button or the Select Data tab
* Click change samples. Ensure only ACS 2016 is selected (You can click the Default sample from each year button to unselect the preselected)
* Select variables
    - STATEFIP, COUNTYFIP, PUMA, DENSITY, NCHILD, SEX, AGE, RACE, HISPAN, EDUC, EMPSTAT, OCC2010, IND1990, UHRSWORK, INCWAGE
* View cart
* For reducing the size of the extract, **unselect** the following preselected variables (we need the others)
    - YEAR, SAMPLE, CBSERIAL, HHWT, CLUSTER, STRATA
* CREATE DATA EXTRACT
* DATA FORMAT > Change > Comma delimited (.csv) > SUBMIT
* Provide a description of the extract like: "AML in Econ homework data"
* SUBMIT EXTRACT

**SUPER IMPORTANT** Each variable (continuous or categorical) has an associated code to it. For an example of a codebook you will need, click on the EDUC variable, go to the codes tab, and select the Detailed codes (44 categories) *and* Case-count view.

To extract:
- **Windows Users**: Use 7zip to extract (like I do) https://www.7-zip.org/download.html
- **MAC Users**: In your terminal, navigate to the location of your file. Type `gunzip usa_000XX.csv.gz`

Because surveys tend to ask a let of yes or no questions, they primarily consist of dummy variables. 
We are going to add to our list of continuous variables by merging county-level housing data from the Federal Housing Finance Agency (FHFA) https://www.fhfa.gov/DataTools/Downloads/Pages/House-Price-Index-Datasets.aspx.

FHFA Steps:

* Download the counties developmental index
* Open it and notice the preamble and where the data begins

***

Because we will be demonstrating different techniques for different types of labels in this class, we will create three different types of label that can be interchangeably used as features. 

* Labels
    - wage (continuous)
    - full-time dummy (binary)
    - degree $\in \{\leq$ HS, some college, $\geq$ BA$\}$
- Features
    * public use micro area (puma) unemployment rate
    * percent of workers in an occupation with a bachelor's degree
    * average industry wages
    * experience = age - years of schooling - 6
    * female indicator
    * any kids at home indicator
    * black indicator
    * hispanic indicator
    * 2000 based house price index

# [10 Points] Preliminaries

- **[5 points]** Import `pandas`, `numpy`, `matplotlib.pyplot`, `seaborn` and `os`.
- **[1 point]** Change your working directory to where your data are saved.
- **[2 points]** List the files in the directory
- **[2 points]** Set up `seaborn` parameters

*******
# ACS
[TOP](#Homework-1---Yeehaw!-Data-Wrangling!)


## [15 Points] Prep
- **[1 point]** Load in the data as an object named `acs`

- **[2 point]** Holding down shift is annoying. Make all of the variables lowercase.

Filter the data set such that:
* **[2 points]** individuals are not in group quarters
* **[2 points]** individuals are working age $\in [16, 65)$
* **[2 points]** there are no military or unemployed with no worth experience in the last five years occupations
* **[2 points]** there are no N/A, military or last worked 1984 or earlier industries
* **[2 points]** there are no N/A education values (*hint: EDUCD*)

* **[2 points]** drop the `gq` column

***
## [5 Points] Creating Keys
[TOP](#Homework-1---Yeehaw!-Data-Wrangling!)

We need to create geographic and individual keys that will be used for linking.
Firstly, create the geographic keys:
* **[1 point]** `county` - county codes
* **[1 point]** `puma_code` - public use micro area code

*Hint: counties are fully identified by a 5 digit code. The first two are the state FIPS and the last three are the county FIPS. puma codes are a 7 digit code like counties. Think of a way using middle school level math to create these codes.*

*For example: Champaign county, IL and Cherokee county, AL both have county FIPS of 019, but IL has state FIPS of 17 and AL has state FIPS 1. This makes Champaign 17019 and Cherokee 1019*

Individuals on the other hand are identified by their household `serial` number combined with the person number (`pernum`) in the household such that the last two digits are the person number. 
- **[1 point]** Using the same methodology as above, create a variable `id`

Now
* **[2 points]** drop columns `countyfip`, `statefip`, `puma`, `pernum`, and `serial` in place

******
## [15 Points] Grouped Features
[TOP](#Homework-1---Yeehaw!-Data-Wrangling!)

Next we will create the following grouped features: (1) unemployment rate grouped by PUMA, (2) percent with a bachelor's degree group by occupation, and (3) average wage grouped by industry. It should take three lines each (fewer if you are a better coder than I!):
1. create an empty temporary data frame
2. create a variable in the temporary data frame from a lambda function applied to the acs grouped by the group variable
3. join the temporary data frame to the ACS data by the group variable

Create the variables as described above
- **[5 points]** `puma_urate`
- **[5 points]** `occ_ba`
- **[5 points]** `ind_wage`

*Hints: Recall that the unemployment rate is given by*
$$unemployment\text{ }rate = 100 \frac{\sum_{i=1}^N I\{unemployed_i\}} {\sum_{i=1}^N I\{labor\text{}force_i\}}$$
*where $I\{unemployed_i\} = 1$ indicates being unemployed and $I\{labor\text{}force_i\} = 1$ indicates being in the labor force.*

*BECAUSE WE HAVE SURVEY DATA, WE NEED INDIVIDUAL WEIGHTS TO CALCULATE AGGREGATE STATISTICS. That means we need to multiply each element in the fraction by* `perwt` *in our code. This also needs to be done in the other two variables.*

*Use logic statements*

In [ ]:
# puma_urate

In [ ]:
# occ_ba

In [ ]:
# ind_wage

****
## [10 Points] Individual Features
[TOP](#Homework-1---Yeehaw!-Data-Wrangling!)

Filter you data set such that
- **[1 point]** all individuals are employed

Next, create the `experience` variable.
To do so, map a dictionary of all detailed education codes to years of schooling `yos`. For clarification: 
* Define one yos to start at 1st grade (set kindergarten and nursery school/preschool to zero).
* Define from 12th grade, no diploma to Some college, but less than one year as 12 yos
* Define bachelor's degree as 16 yos
* Define master's degree as 18 yos
* Define professional/doctoral degree as 20 yos

One you have mapped the dictionary to the detailed education variable, create the experience variable:
* **[5 points]** `experience` = `age` - `yos` - 6

Finally, create the dummy features for:

* **[1 point]** `female`
* **[1 point]** `any_kids`
* **[1 point]** `black`
* **[1 point]** `hispanic`

*******
## [5 Points] Labels
[TOP](#Homework-1---Yeehaw!-Data-Wrangling!)

Now to create our three different labels:
1. **[1 points]** `full_time` - works weakly greater than 35 hours
    - *hint: logic statement made integer*
2. **[2 points]** `degree` with values `hs or less`, `some college`, and `ba or more`
    - *hint: use a nested* `np.where()`
3. **[0 points]** `incwage`
    - *hint: this hint is intentionally left blank, but by leaving a message it is now no longer blank*

Here are the variables we need: `id`, `county`, `occ2010`, `ind1990`, `incwage`, `full_time`, `degree`, `density`, `experience`, `puma_urate`, `occ_ba`, `ind_wage`, `female`, `any_kids`, `black`, `hispanic`

* **[1 point]** Trim the `acs` data frame by explicitly selecting the variables we need
* **[1 point]** print the shape of `acs`

*** 
# [10 Points] FHFA
[TOP](#Homework-1---Yeehaw!-Data-Wrangling!)

* **[2 points]** Load in the HPI data as `hpi` taking into account the preamble.

When you get a message that says you need the package xlrd and then you are like okay, I'll go install that and then you run the code again and then you get the message the xlsx file not support and you are like w(hy)tf not?! specify the option `engine = 'openpyxl'` inside of the pandas read function.

* **[2 points]** Make the variables lower case

In one line
* **[4 points]** filter to only have 2016 data, select the FIPS code and the 2000-based hpi, and rename the selected columns with a dictionary to be `county` and `hpi` respectively

In one line
* **[2 points]** create the data frame `df` using an inner merge with `acs` and `hpi` on the appropriate key variable and then set the index to `id`

***** 
# [25 Points] Transformations
[TOP](#Homework-1---Yeehaw!-Data-Wrangling!)

By this point, you should have **wrangled** the data set for *all* subsequent homework assignments.
Now it is time to clean it and deal with any necessary (logarithm) transformations.

* **[1 point]** print the info from `df`

One variable should not be string. 
* **[1 point]** identify which variable should not be a string and convert it as a float type
* **[4 points]** read the error and drop the troublesome value(s), then convert the variable as a float type
* **[2 point]** print the info again to show you have handled it

***
Here is a helping hand. 
Only two variables need a log transformation.

For the two identified variables
* **[10 points, 1 per item over 2 figures]** produce a figure with 
    - two side-by-side histograms where the left is the raw data and the left is the data on a log axis
    - with a tight layout
    - with appropriate titles and x-axis labels
    - adjust the number of bins (say 10)
    - *hint: what is log(0)?*
* **[2 points]** drop any observations with troublesome values
* **[4 points]** create a new variable with the prefix `l_` and drop the original 
    - `julian` $\rightarrow$ `l_julian`


In [ ]:
# Drop

In [ ]:
# Figure 1

In [ ]:
# Variable 1

In [ ]:
# Figure 2

In [ ]:
# Variable 2

* **[1 point]** reorder the variables matching the `acs` selection above with the transformed variables
    - Order: keys, labels, continuous features, discrete features
    - `hpi` wink, wink

*********
# [5 Points] Pickling
[TOP](#Homework-1---Yeehaw!-Data-Wrangling!)

* **[2 point]** First, print the head of your data frame.

* **[3 points]** Save the final data frame to a pickle with a fancy name like "hw_data.pkl"